<a href="https://colab.research.google.com/github/maliksh7/Anomaly-Detection-System/blob/main/Anomaly_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly Detection System (IDS)
### Models:
- Decision Tree Classifier
- Random Forest
- Extra Tree
- Naive Bayes
- Logistic Regression
- Multi-Layered Perceptron (MLP)

### Dataset:
- used CIC 2017 and 2018 Datasets
- 19Lac Big training data samples
- 800K Validation data samples

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import (
    BatchNormalization, Input,Dense,Flatten,Dropout,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
)

from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers

from keras import backend as K
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Final Year Project/fyp-2/Datasets/MachineLearningCVE/19LAC-Big-Training.csv')

In [ ]:

df[' Label'].value_counts()

0     1500000
10     100000
4      100000
2      100000
1      100000
Name:  Label, dtype: int64

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Dropping all the rows with nan values
df.dropna(inplace=True)

In [ ]:
del df['Unnamed: 0']
df = df.astype('uint8', errors='ignore')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1801956 entries, 0 to 1899999
Data columns (total 55 columns):
 #   Column                       Dtype
---  ------                       -----
 0    Destination Port            uint8
 1    Flow Duration               uint8
 2    Total Fwd Packets           uint8
 3    Total Backward Packets      uint8
 4   Total Length of Fwd Packets  uint8
 5    Fwd Packet Length Max       uint8
 6    Fwd Packet Length Min       uint8
 7    Fwd Packet Length Mean      uint8
 8    Fwd Packet Length Std       uint8
 9   Bwd Packet Length Max        uint8
 10   Bwd Packet Length Min       uint8
 11   Bwd Packet Length Mean      uint8
 12   Bwd Packet Length Std       uint8
 13  Flow Bytes/s                 uint8
 14   Flow Packets/s              uint8
 15   Flow IAT Mean               uint8
 16   Flow IAT Std                uint8
 17   Flow IAT Max                uint8
 18   Flow IAT Min                uint8
 19  Fwd IAT Total                uint8
 20   F

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y=le.fit_transform(df[' Label'])
# len(Y)
X=df.iloc[:,:-1]

In [ ]:
X=df.iloc[:,:-1]

print(X.shape)
X.dropna()
print(X.shape,np.max(np.max(X).astype(np.float32)))
count = -1
for i in np.max(X).astype(np.float32):
    count += 1
    print(count, '---',i)

(1801956, 54)
(1801956, 54) 255.0
0 --- 255.0
1 --- 255.0
2 --- 255.0
3 --- 255.0
4 --- 255.0
5 --- 255.0
6 --- 254.0
7 --- 255.0
8 --- 255.0
9 --- 255.0
10 --- 255.0
11 --- 255.0
12 --- 255.0
13 --- 255.0
14 --- 255.0
15 --- 255.0
16 --- 255.0
17 --- 255.0
18 --- 255.0
19 --- 255.0
20 --- 255.0
21 --- 255.0
22 --- 255.0
23 --- 255.0
24 --- 255.0
25 --- 255.0
26 --- 1.0
27 --- 252.0
28 --- 252.0
29 --- 255.0
30 --- 254.0
31 --- 255.0
32 --- 255.0
33 --- 255.0
34 --- 255.0
35 --- 1.0
36 --- 1.0
37 --- 1.0
38 --- 1.0
39 --- 255.0
40 --- 255.0
41 --- 255.0
42 --- 255.0
43 --- 255.0
44 --- 255.0
45 --- 255.0
46 --- 255.0
47 --- 255.0
48 --- 255.0
49 --- 255.0
50 --- 255.0
51 --- 255.0
52 --- 255.0
53 --- 255.0


In [ ]:
np.random.seed(0)
#x_train, x_test, y_train, y_test = train_test_split(label_puts, target, test_size = 0.99, random_state = 42)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
print(x_train.shape,len(y_train),x_test.shape,len(y_test))
x_train.columns

(1441564, 54) 1441564 (360392, 54) 360392


Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Max',
       'Fwd PSH Flags', ' Fwd Header Length', ' Bwd Header Length',
       'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' PSH Flag Count', ' ACK Flag Count',
       ' URG Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size',
       ' Avg Bwd Segment Size', 'Subf

In [ ]:
df[' Label'].value_counts()

0     1500000
10     100000
4      100000
2      100000
1        1956
Name:  Label, dtype: int64

In [ ]:
df.shape

(1801956, 55)

In [ ]:
filename = ('/content/gdrive/MyDrive/Final Year Project/fyp-2/Datasets/MachineLearningCVE/18LAC-Big-Training.h5')
df.to_hdf(filename, 'data', mode='w', format='table')

In [ ]:
df.shape

(1801956, 55)

In [ ]:
from pandas.io.pytables import read_hdf
df = read_hdf(filename)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1801956 entries, 0 to 1899999
Data columns (total 55 columns):
 #   Column                       Dtype
---  ------                       -----
 0    Destination Port            uint8
 1    Flow Duration               uint8
 2    Total Fwd Packets           uint8
 3    Total Backward Packets      uint8
 4   Total Length of Fwd Packets  uint8
 5    Fwd Packet Length Max       uint8
 6    Fwd Packet Length Min       uint8
 7    Fwd Packet Length Mean      uint8
 8    Fwd Packet Length Std       uint8
 9   Bwd Packet Length Max        uint8
 10   Bwd Packet Length Min       uint8
 11   Bwd Packet Length Mean      uint8
 12   Bwd Packet Length Std       uint8
 13  Flow Bytes/s                 uint8
 14   Flow Packets/s              uint8
 15   Flow IAT Mean               uint8
 16   Flow IAT Std                uint8
 17   Flow IAT Max                uint8
 18   Flow IAT Min                uint8
 19  Fwd IAT Total                uint8
 20   F

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y=le.fit_transform(df[' Label'])
len(Y)
X=df.iloc[:,:-1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
print(x_train.shape,len(y_train),x_test.shape,len(y_test))
len(x_train.columns)

(1441564, 54) 1441564 (360392, 54) 360392


54

#**Decision Tree Classifier**

In [ ]:
print(x_train.shape, y_train.shape)
dtc = DecisionTreeClassifier(max_depth = 15)
dtc = dtc.fit(x_train,y_train)

y_pred = dtc.predict(x_test)
y_pred

from sklearn.metrics import accuracy_score,f1_score
print(dtc.score(x_test, y_test))
print('Accuracy', accuracy_score(y_test, y_pred))
print('F1 Score', f1_score(y_test, y_pred, average='macro'))

(1441564, 54) (1441564,)
0.9986264955936869
Accuracy 0.9986264955936869
F1 Score 0.9569747099218142


In [ ]:
validation_accuracy = []
train_accuracy = []

for depth in range(1,15):
    dt_model = DecisionTreeClassifier(max_depth=depth, random_state=10)
    dt_model.fit(x_train, y_train)
    train_accuracy.append(dt_model.score(x_train, y_train))
    validation_accuracy.append(dt_model.score(x_test, y_test))
print(len(train_accuracy))
print(len(validation_accuracy))
frame = pd.DataFrame({'max_depth':range(1,15), 'train_acc':train_accuracy, 'valid_acc':validation_accuracy})

14
14


In [ ]:
frame

,max_depth,train_acc,valid_acc
0,1,0.832471,0.832260
1,2,0.877835,0.877339
2,3,0.895412,0.894828
3,4,0.932003,0.931186
4,5,0.970905,0.970690
5,6,0.978472,0.978213
6,7,0.992508,0.992408
7,8,0.998103,0.997997
8,9,0.998500,0.998418
9,10,0.998615,0.998485


#**Random Forest CLassifier**

In [ ]:
# print('---------------------------Random Forest CLassifier------------------')
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators =5)
model = model.fit(x_train, y_train)
rfy_pred = model.predict(x_test)
acc = metrics.accuracy_score(y_test, rfy_pred)
print('F1 Score: ', f1_score(y_test, rfy_pred, average='macro'))
print('Accuracy: ', acc)
print(model.score(x_test,y_test))

F1 Score:  0.9474881610775279
Accuracy:  0.9982491287264978
0.9982491287264978


#**Extra Trees Classifier**

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(x_train,y_train)

ExtraTreesClassifier()

In [ ]:
rfy_pred = model.predict(x_test)
acc = metrics.accuracy_score(y_test, rfy_pred)
print('Accuracy: ', acc)
print('F1 Score: ', f1_score(y_test, rfy_pred, average='macro'))
print(model.score(x_test,y_test))

Accuracy:  0.9981742103043353
F1 Score:  0.9459132325287506
0.9981742103043353


#**Gaussian Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(x_train, y_train)
rfy_pred = gnb.predict(x_test)


In [ ]:
acc = metrics.accuracy_score(y_test, rfy_pred)
print('Accuracy: ', acc)
print('F1 Score: ', f1_score(y_test, rfy_pred, average='macro'))
print(model.score(x_test,y_test))

Accuracy:  0.7471752980088349
F1 Score:  0.7319052563191544
0.9981742103043353


#**Linear Regression**

In [ ]:
clf_logistic = LogisticRegression()
clf_logistic = clf_logistic.fit(x_train, y_train)

rfy_pred = clf_logistic.predict(x_test)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# acc = metrics.accuracy_score(y_test, rfy_pred)
print('F1 Score: ', f1_score(y_test, rfy_pred, average='macro'))
print('Score: ',clf_logistic.score(x_test,y_test))

F1 Score:  0.7762083784593259
Score:  0.9715171257963551


#**Multi-Layered Perceptron (MLP)**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Final Year Project/fyp-2/Datasets/MachineLearningCVE/20LAC-Big-Training.csv')

In [ ]:
df[' Label'].value_counts()

0     1500000
14     100000
8      100000
7      100000
4      100000
2      100000
1        1000
Name:  Label, dtype: int64

In [ ]:
# del df['Unnamed: 0.1']
# del df['Unnamed: 0']

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001000 entries, 0 to 2000999
Data columns (total 55 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0    Destination Port            float64
 1    Flow Duration               float64
 2    Total Fwd Packets           float64
 3    Total Backward Packets      float64
 4   Total Length of Fwd Packets  float64
 5    Fwd Packet Length Max       float64
 6    Fwd Packet Length Min       float64
 7    Fwd Packet Length Mean      float64
 8    Fwd Packet Length Std       float64
 9   Bwd Packet Length Max        float64
 10   Bwd Packet Length Min       float64
 11   Bwd Packet Length Mean      float64
 12   Bwd Packet Length Std       float64
 13  Flow Bytes/s                 float64
 14   Flow Packets/s              float64
 15   Flow IAT Mean               float64
 16   Flow IAT Std                float64
 17   Flow IAT Max                float64
 18   Flow IAT Min                float64
 19  

In [ ]:
df[' Label'].value_counts()


0     1500000
14     100000
8      100000
7      100000
4      100000
2      100000
1        1000
Name:  Label, dtype: int64

In [ ]:
def dpna(df):
    # df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Dropping all the rows with nan valuess
    df.dropna(inplace=True)
    return df

# -------------------checking datatypes

def check_size_dtypes(df):

    max = df.max()
    print(max, 'max')

    min = df.min()
    print(min, 'min')
    # print(df.value_counts())
    var1 = df.memory_usage(index=False, deep=True)
    print(var1, 'This is the memory usage')
    print(df.sample(8))

# -------------------changing datatypes

def convert_datatypes(df, a='uint8'):
    print('Trying to convert datatypes for less memory usage')
    max = df.max()
    print(max, 'max')

    min = df.min()
    print(min, 'min')

    # print(df.value_counts())

    var1 = df.memory_usage(index=False, deep=True)
    print(var1, 'memory usage')
    df = df.astype(a, errors='ignore')
    print("[ ----- Changed Datatypes to 'uint8' ----- ]")
    var2 = df.memory_usage(index=False, deep=True)
    print(var2, ' new memory usage| the difference -> ', var1 / var2)
    return df

def normalize(df):
    print("[* ] - Normalized data")
    normalized_df = ((df - df.min()) /
                      (df.max() - df.min())) * 225
    return normalized_df

In [ ]:
# dpna(df)
# check_size_dtypes(df)
# convert_datatypes(df, a='uint8')
# normalize(df)
df = df.astype('uint8', errors='ignore')

In [ ]:
df[' Label'].value_counts()

0     1500000
14     100000
8      100000
7      100000
4      100000
2      100000
1        1000
Name:  Label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

# df.replace([np.inf, -np.inf], np.nan, inplace=True)
# # Dropping all the rows with nan values
# df.dropna(inplace=True)

le = LabelEncoder()
Y=df[' Label']
# len(Y)
X=df.iloc[:,:-1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.35, random_state = 42)
print(x_train.shape,len(y_train),x_test.shape,len(y_test))
len(x_train.columns)

(1300650, 54) 1300650 (700350, 54) 700350


54

In [ ]:
df[' Label'].value_counts()

0     1500000
14     100000
8      100000
7      100000
4      100000
2      100000
1        1000
Name:  Label, dtype: int64

In [ ]:
df.shape

(2001000, 55)

In [ ]:
print(y_train.shape, y_test.shape)

(1171271,) (630685,)


In [ ]:
# Imports
import tensorflow
# from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

# Configuration options
feature_vector_length = 55
num_classes = 7



Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)



In [ ]:
num_classes

4

In [ ]:
# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()
model.add(Input(shape=(55 )))

model.add(Dense(28, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(14, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))



Feature shape: (55,)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 28)                1568      
                                                                 
 dropout (Dropout)           (None, 28)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                406       
                                                                 
 dropout_1 (Dropout)         (None, 14)                0         
                                                                 
 dense_2 (Dense)             (None, 7)                 105       
                                                                 
 dropout_2 (Dropout)         (None, 7)                 0         
                                                                 
 dense_3 (Dense)             (None, 4)                 3

In [ ]:
type(x_train)

pandas.core.frame.DataFrame

In [ ]:
# Configure the model and start training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
model.fit(x_train, Y_train, epochs=100, batch_size=64, verbose=1, validation_data=(x_test, Y_test))

# Test the model after training
test_results = model.evaluate(x_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

Epoch 1/100
8125/8125 [==============================] - 20s 2ms/step - loss: 0.1034 - accuracy: 0.9702 - mse: 0.0116 - val_loss: 0.0392 - val_accuracy: 0.9916 - val_mse: 0.0034
Epoch 2/100
8125/8125 [==============================] - 20s 2ms/step - loss: 0.0932 - accuracy: 0.9732 - mse: 0.0104 - val_loss: 0.0365 - val_accuracy: 0.9927 - val_mse: 0.0031
Epoch 3/100
8125/8125 [==============================] - 22s 3ms/step - loss: 0.0871 - accuracy: 0.9750 - mse: 0.0098 - val_loss: 0.0503 - val_accuracy: 0.9939 - val_mse: 0.0026
Epoch 4/100
8125/8125 [==============================] - 21s 3ms/step - loss: 0.0865 - accuracy: 0.9752 - mse: 0.0097 - val_loss: 0.0291 - val_accuracy: 0.9934 - val_mse: 0.0026
Epoch 5/100
8125/8125 [==============================] - 22s 3ms/step - loss: 0.0808 - accuracy: 0.9774 - mse: 0.0089 - val_loss: 0.0250 - val_accuracy: 0.9935 - val_mse: 0.0026
Epoch 6/100
8125/8125 [==============================] - 20s 2ms/step - loss: 0.0779 - accuracy: 0.9782 - mse:

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520000 entries, 2253029 to 250032
Data columns (total 55 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0    Destination Port            520000 non-null  uint8
 1    Flow Duration               520000 non-null  uint8
 2    Total Fwd Packets           520000 non-null  uint8
 3    Total Backward Packets      520000 non-null  uint8
 4   Total Length of Fwd Packets  520000 non-null  uint8
 5    Fwd Packet Length Max       520000 non-null  uint8
 6    Fwd Packet Length Min       520000 non-null  uint8
 7    Fwd Packet Length Mean      520000 non-null  uint8
 8    Fwd Packet Length Std       520000 non-null  uint8
 9   Bwd Packet Length Max        520000 non-null  uint8
 10   Bwd Packet Length Min       520000 non-null  uint8
 11   Bwd Packet Length Mean      520000 non-null  uint8
 12   Bwd Packet Length Std       520000 non-null  uint8
 13  Flow Bytes/s           

In [ ]:
new_df[' Label'].value_counts()

In [ ]:
test_results = model.evaluate(x_test, Y_test, verbose=1)

8750/8750 [==============================] - 15s 2ms/step - loss: 0.0222 - accuracy: 0.9941 - mse: 0.0021


In [ ]:
x_test.head(10)

In [ ]:
l = model.predict(x_test, verbose=0)

In [ ]:
l = np.argmax(l, axis=1)
prediction_int = np.zeroes_like(prediction)
prediction_int[prediction > 0.5] = 1

In [ ]:
print(l)
l = l.round()
print(l)

[[9.9999988e-01 9.4183157e-08 4.1691074e-11 0.0000000e+00]
 [1.0000000e+00 6.2043266e-11 4.0033752e-16 0.0000000e+00]
 [1.0000000e+00 1.6741014e-10 1.0743874e-15 0.0000000e+00]
 ...
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]
 [1.0000000e+00 5.3965986e-21 3.7108573e-32 0.0000000e+00]
 [1.8769801e-05 7.1809183e-17 9.9998128e-01 0.0000000e+00]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [ ]:
from sklearn.metrics import f1_score


In [ ]:
print(f1_score(Y_test, l, average='macro'), 'f1') 
# 0.22442756164188604 f1


0.992935546000768 f1


In [ ]:
l

array([[0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       ...,
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ],
       [0.8108735 , 0.04763747, 0.08484388, 0.0566451 ]], dtype=float32)

In [ ]:
Y_test

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [ ]:
filename = ('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/800kValidation-Dos-DDos-PS-Benign.h5')
valid_df = pd.read_hdf(filename)

In [ ]:
del valid_df['Unnamed: 0']
valid_df.columns

In [ ]:
valid_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Dropping all the rows with nan values
valid_df.dropna(inplace=True)

# le = LabelEncoder()
Y=valid_df[' Label']
# len(Y)
X=valid_df.iloc[:,:-1]

In [ ]:
# Y_train = to_categorical(, num_classes)
Y_test = to_categorical(Y, num_classes)

In [ ]:
check = model.predict(X, verbose=0)

In [ ]:
print(check)
check = check.round()
print(check)

[[1.0000000e+00 1.3289315e-13 2.5395361e-20 0.0000000e+00]
 [1.0000000e+00 8.0662291e-12 2.2246202e-17 0.0000000e+00]
 [1.0000000e+00 3.3879440e-21 9.8526837e-35 0.0000000e+00]
 ...
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]
 [1.4764822e-02 8.7535969e-04 3.3552384e-03 9.8100454e-01]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [ ]:
print(f1_score(Y_test, check, average='macro'), 'f1') 
# 0.22442756164188604 f1


0.99187354905695 f1


In [ ]:
filename = ('/content/drive/MyDrive/MNAD-FYP/MachineLearningCVE/Dataset-Analysis/MNAD-DNN.h5')
# model.save_weights(filename)